In [99]:
import pandas as pd
import numpy as np

In [146]:
# GDP and military data
data = pd.read_excel(r'C:\Users\admin\Downloads\Popular Indicators.xlsx')

# education data
edu_data = pd.read_excel(r'C:\Users\admin\Desktop\Individual-Project\Education Expenditure.xls')

# healthcare data
health_data = pd.read_excel(r'Health Expenditure.xls')


In [11]:
data.columns

Index(['Series Name', 'Series Code', 'Country Name', 'Country Code',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]',
       '2019 [YR2019]', '2020 [YR2020]'],
      dtype='object')

In [175]:
#  G20 countries
G20Countries = ['Argentina', 'Australia', 'Brazil', 'Canada', 
'China', 'France', 'Germany', 'India', 'Indonesia', 'Italy', 
'Japan', 'Korea, Rep.', 'Mexico', 'Russian Federation', 'Saudi Arabia', 
'South Africa', 'Turkey', 'United Kingdom', 'United States', 'European Union']


# Series Name 
# NY.GDP.MKTP.CD is GDP current USD
# MS.MIL.XPND.GD.ZS IS Military expenditure (% of GDP)
# NY.GDP.PCAP.CD is GDP per capita (current US$)
# population is SP.POP.TOTL
series_list = ['GDP (current US$)', 'Military expenditure (% of GDP)', 'GDP per capita (current US$)', 'Population, total']

In [184]:
def get_gdp_data(data):
# filter for countries
    sub_data = data[data['Country Name'].isin(G20Countries)]
    sub_data.columns = ['Series Name', 'Series Code', 'Country Name', 'Country Code', '2011',
           '2012', '2013', '2014', '2015',
           '2016', '2017', '2018', '2019',
           '2020']
    processed_df = pd.DataFrame()
    for idx in range(len(series_list)):
        temp = sub_data[sub_data['Series Name'] == series_list[idx]]
        pivoted = pd.melt(temp, id_vars='Country Name', value_vars=['2011','2012', '2013', '2014', '2015',
           '2016', '2017', '2018', '2019'], value_name=series_list[idx])
        if idx > 0:
            temp_2 = pd.merge(temp_2, pivoted, on =['Country Name', 'variable'])
        else:
            temp_2 = pivoted
    temp_2 = temp_2.replace('..', np.NaN)
    temp_2 = temp_2.groupby('Country Name').apply(lambda group: group.interpolate(method='linear'))
    return temp_2

gdp_data = get_gdp_data(data)
gdp_data.columns = ['Country Name', 'Year', 'GDP (current US$)',
       'Military expenditure (% of GDP)', 'GDP per capita (current US$)',
       'Population, total']

In [178]:
def transform_edu_data(df):
    sub_df = df[df['Country Name'].isin(G20Countries)]
#     print(sub_df)
    pivoted = pd.melt(sub_df, id_vars='Country Name', value_vars=['2011','2012', '2013', '2014', '2015',
           '2016', '2017', '2018', '2019'])
#     temp_2 = pivoted.replace('..', np.NaN)
    pivoted = pivoted.groupby('Country Name').apply(lambda group: group.interpolate(method='index'))
    pivoted = pivoted.groupby("Country Name").apply(lambda x: x.fillna(method='bfill'))
    return pivoted
edu_data_processed = transform_edu_data(edu_data)
edu_data_processed.columns = ['Country Name', 'Year', 'Government expenditure on education, total (% of GDP)']
edu_data_processed

,Country Name,Year,"Government expenditure on education, total (% of GDP)"
0,Argentina,2011,5.29063
1,Australia,2011,5.07451
2,Brazil,2011,5.73741
3,Canada,2011,5.27444
4,China,2011,NaN
...,...,...,...
175,Russian Federation,2019,3.74303
176,Saudi Arabia,2019,NaN
177,Turkey,2019,NaN
178,United States,2019,4.96174


In [140]:
edu_data_processed[edu_data_processed['Government expenditure on education, total (% of GDP)'].isna()
                  ].sort_values(['Country Name','Year'])

,Country Name,Year,"Government expenditure on education, total (% of GDP)"
4,China,2011,NaN
24,China,2012,NaN
44,China,2013,NaN
64,China,2014,NaN
84,China,2015,NaN
104,China,2016,NaN
124,China,2017,NaN
144,China,2018,NaN
164,China,2019,NaN
7,France,2011,NaN


In [174]:
healthcare_list = ['Current health expenditure per capita, PPP (current international $)', 'Current health expenditure (% of GDP)']
def healthcare_data(df):
    sub_data = df[df['Country Name'].isin(G20Countries)]
    for idx in range(len(healthcare_list)):
        temp = sub_data[sub_data['Indicator Name'] == healthcare_list[idx]]
        pivoted = pd.melt(temp, id_vars='Country Name', value_vars=['2011','2012', '2013', '2014', '2015',
               '2016', '2017', '2018', '2019'], value_name=healthcare_list[idx])
        if idx > 0:
            temp_2 = pd.merge(temp_2, pivoted, on =['Country Name', 'variable'])
        else:
            temp_2 = pivoted
    temp_2 = temp_2.groupby('Country Name').apply(lambda group: group.interpolate(method='index'))
    temp_2 = temp_2.groupby("Country Name").apply(lambda x: x.fillna(method='bfill'))
    return temp_2

health_data_processed = healthcare_data(health_data)
health_data_processed.columns = ['Country Name', 'Year', 
                                 'Current health expenditure per capita, PPP (current international $)',
                                'Current health expenditure (% of GDP)']

In [186]:
final_data = pd.merge(gdp_data, health_data_processed, on=['Country Name', 'Year'])
final_data = pd.merge(final_data, edu_data_processed, on=['Country Name', 'Year'])
final_data.head()

,Country Name,Year,GDP (current US$),Military expenditure (% of GDP),GDP per capita (current US$),"Population, total","Current health expenditure per capita, PPP (current international $)",Current health expenditure (% of GDP),"Government expenditure on education, total (% of GDP)"
0,European Union,2011,1.574184e+13,1.501848,35716.273663,440746976,3297.408614,NaN,5.01738
1,Argentina,2011,5.301633e+11,0.764287,12848.864197,41261490,1648.187744,8.396563,5.29063
2,Australia,2011,1.396650e+12,1.769390,62517.833747,22340024,3770.488281,8.541520,5.07451
3,Brazil,2011,2.616201e+12,1.411851,13245.612500,197514534,1166.133545,7.788301,5.73741
4,Canada,2011,1.788648e+12,1.196084,52087.446384,34339328,4278.646973,10.352504,5.27444


In [189]:
# (final_data['Current health expenditure (% of GDP)'] * final_data['GDP (current US$)'])/final_data['Population, total']
final_data['Military expenditure (current US$)'] = final_data['Military expenditure (% of GDP)'] * final_data['GDP (current US$)']
final_data['Education expenditure (current US$)'] = final_data['Government expenditure on education, total (% of GDP)'] * final_data['GDP (current US$)']
final_data['Health expenditure (current US$)'] = final_data['Current health expenditure (% of GDP)'] * final_data['GDP (current US$)']

# per capita
final_data['Military expenditure per capita'] = final_data['Military expenditure (current US$)'] / final_data['Population, total']
final_data['Education expenditure per capita'] = final_data['Education expenditure (current US$)'] / final_data['Population, total']
final_data['Health expenditure per capita'] = final_data['Health expenditure (current US$)'] / final_data['Population, total']

In [190]:
final_data.head()

,Country Name,Year,GDP (current US$),Military expenditure (% of GDP),GDP per capita (current US$),"Population, total","Current health expenditure per capita, PPP (current international $)",Current health expenditure (% of GDP),"Government expenditure on education, total (% of GDP)",Military expenditure (current US$),Education expenditure (current US$),Health expenditure (current US$),Military expenditure per capita,Education expenditure per capita,Health expenditure per capita
0,European Union,2011,1.574184e+13,1.501848,35716.273663,440746976,3297.408614,NaN,5.01738,2.364185e+13,7.898279e+13,NaN,53640.406211,179202.117151,NaN
1,Argentina,2011,5.301633e+11,0.764287,12848.864197,41261490,1648.187744,8.396563,5.29063,4.051970e+11,2.804898e+12,4.451549e+12,9820.221079,67978.586386,107886.296859
2,Australia,2011,1.396650e+12,1.769390,62517.833747,22340024,3770.488281,8.541520,5.07451,2.471218e+12,7.087314e+12,1.192951e+13,110618.421360,317247.372528,533997.323551
3,Brazil,2011,2.616201e+12,1.411851,13245.612500,197514534,1166.133545,7.788301,5.73741,3.693686e+12,1.501022e+13,2.037576e+13,18700.833362,75995.509611,103160.820510
4,Canada,2011,1.788648e+12,1.196084,52087.446384,34339328,4278.646973,10.352504,5.27444,2.139374e+12,9.434116e+12,1.851698e+13,62300.985077,274732.110703,539235.471426


In [193]:
final_data.to_csv('final_data.csv')

In [194]:
final_data.isnull().sum()

Country Name                                                             0
Year                                                                     0
GDP (current US$)                                                        0
Military expenditure (% of GDP)                                          0
GDP per capita (current US$)                                             0
Population, total                                                        0
Current health expenditure per capita, PPP (current international $)     0
Current health expenditure (% of GDP)                                    9
Government expenditure on education, total (% of GDP)                   36
Military expenditure (current US$)                                       0
Education expenditure (current US$)                                     36
Health expenditure (current US$)                                         9
Military expenditure per capita                                          0
Education expenditure per

In [192]:
health_data_processed[health_data_processed['Current health expenditure (% of GDP)'].isna()
                  ].sort_values(['Country Name','Year'])

,Country Name,Year,"Current health expenditure per capita, PPP (current international $)",Current health expenditure (% of GDP)
6,European Union,2011,3297.408614,NaN
26,European Union,2012,3371.177646,NaN
46,European Union,2013,3509.424214,NaN
66,European Union,2014,3607.993827,NaN
86,European Union,2015,3699.950376,NaN
106,European Union,2016,3850.286104,NaN
126,European Union,2017,4054.274484,NaN
146,European Union,2018,4054.274484,NaN
166,European Union,2019,4054.274484,NaN
